In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import firedrake
from firedrake import inner, div, dx, ds, sqrt
import icepack, icepack.models, icepack.plot

# Synthetic ice stream

In this demo, we'll simulate the evolution of an ice stream with both grounded and floating parts.
Ice streams are substantially more complicated than a floating ice shelf.
First, there are more fields.
For floating ice shelves, we only had to consider the velocity, thickness, and fluidity, but for ice streams there's also the surface elevation and bed friction.
Second, the position of the grounded line -- the contour across which the glacier is thin enough to go afloat -- can migrate in time.

The diagnostic equation for an ice stream is similar to that of a floating ice shelf, but for the addition of a term for basal friction:

$$\nabla\cdot hM - C|u|^{1/m - 1}u - \rho gh\nabla s = 0,$$

where $m$ is the *sliding exponent* and $C$ is the *sliding coefficient*.

The glacier state we'll start out with is grounded throughout the entire domain, and during the simulation it will thin out and go afloat part way through.
Accurately predicting grounding line migration is a major problem in glacier flow modeling.

### Geometry and input data

We'll use an elongated fjord-like geometry, 12 km wide and 50 km from the inflow boundary to the ice front.
The bedrock slopes down from 200 m above sea level at the inflow boundary to -400 m at the terminus; the glacier becomes marine one third of the way into the domain.

In [ ]:
import numpy as np
from numpy import pi as π
import pygmsh
R = 200e3
δx = 10e3
θ = π / 3
ϕ = 5 * π / 24

x1 = np.array([-1, 0, 0])
x2 = np.array([+1, 0, 0])
y1 = np.array([-np.cos(θ), np.sin(θ), 0])
y2 = np.array([+np.cos(θ), np.sin(θ), 0])
c1 = 2 * y1
c2 = 2 * y2
z1 = c1 + np.array([np.cos(θ - ϕ), -np.sin(θ - ϕ), 0])
z2 = c2 - np.array([np.cos(θ - ϕ), +np.sin(θ - ϕ), 0])

fig, axes = plt.subplots()
axes.plot(x1[0], x1[1], 'o', label='x1')
axes.plot(y1[0], y1[1], 'o', label='y1')
axes.plot(z1[0], z1[1], 'o', label='z1')
axes.plot(x2[0], x2[1], 'o', label='x2')
axes.plot(y2[0], y2[1], 'o', label='y2')
axes.plot(z2[0], z2[1], 'o', label='z2')
axes.plot(c1[0], c1[1], '+', label='c1')
axes.plot(c2[0], c2[1], '+', label='c2')
axes.legend()

In [ ]:
geometry = pygmsh.built_in.Geometry()

center0 = geometry.add_point([0, 0, 0], lcar=δx)
center1 = geometry.add_point(R * c1, lcar=δx)
center2 = geometry.add_point(R * c2, lcar=δx)
center3 = geometry.add_point([0, +4 * R, 0], lcar=δx)

x1 = geometry.add_point(R * x1, lcar=δx)
x2 = geometry.add_point(R * x2, lcar=δx)
y1 = geometry.add_point(R * y1, lcar=δx)
y2 = geometry.add_point(R * y2, lcar=δx)
z1 = geometry.add_point(R * z1, lcar=δx)
z2 = geometry.add_point(R * z2, lcar=δx)

arcs = [geometry.add_circle_arc(x1, center3, x2),
        geometry.add_circle_arc(x2, center0, y2),
        geometry.add_circle_arc(y2, center2, z2),
        geometry.add_line(z2, z1),
        geometry.add_circle_arc(z1, center1, y1),
        geometry.add_circle_arc(y1, center0, x1)]

line_loop = geometry.add_line_loop(arcs)
plane_surface = geometry.add_plane_surface(line_loop)
physical_lines = [geometry.add_physical(arc) for arc in arcs]
physical_surface = geometry.add_physical(plane_surface)

with open('ice-stream.geo', 'w') as geo_file:
    geo_file.write(geometry.get_code())
    
!gmsh -2 -format msh2 -o ice-stream.msh ice-stream.geo

In [ ]:
mesh = firedrake.Mesh('ice-stream.msh')

In [ ]:
fig, axes = icepack.plot.subplots()
icepack.plot.triplot(mesh)

In [ ]:
Q = firedrake.FunctionSpace(mesh, family='CG', degree=2)
V = firedrake.VectorFunctionSpace(mesh, family='CG', degree=2)

In [ ]:
x, y = firedrake.SpatialCoordinate(mesh)

b_in, b_out = 200, -400

As a sanity check, we'll evaluate the driving stress

$$\tau_D = -\rho_Igh\nabla s$$

at the inflow boundary of the ice stream to make sure the value isn't absurd.